# 2017-04-09
----
Reminders : I'm trying to write a pythn-on script that run as background (daemon) on the Pi, that allow to interact with few command lines (API, maybe using zmq and/or socket) to do 3 tasks :
- Ask the camera if it's ready to record
- Allow the camera to answer positively or negatively
- Ask the camera to cut down the connection  


- [ ] Create a daemon

Like last week, I have connection problems to the Pi. But this time, restarting it didn't solved the problem. After talking with L. Perrinet, we didn't found where it come from.  
I'll work on the daemon/API script until further notice.  
More informations about zmq : http://zeromq.org/, http://api.zeromq.org/ and http://zguide.zeromq.org/py:all  
Going further with the interactive python scripts : https://docs.python.org/2/library/thread.html#module-thread and https://docs.python.org/2/library/threading.html  

The daemon script is under construction : pi_interaction.ipynb

The connection to the Pi works with a new IP : __10.164.7.213__

    ssh pi@10.164.7.213
